In [41]:
!pip install transformers torch


In [52]:
!ngrok config add-authtoken 2wfwWFsAfBFbmG5GEGLUzxWBFdr_FHAHhGyqWucqugh4wwL2


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [42]:

!pip install pyngrok


In [43]:
!pip install flask-ngrok


In [44]:
!pip install gradio

In [45]:
!pip install emoji

In [46]:
!pip install -q grpcio grpcio-tools


In [47]:
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. story_service.proto


In [48]:
%pip install -q google-colab-selenium

In [ ]:
import os
import time
import emoji
import asyncio
import json
import threading
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from flask import Flask, request, jsonify
import gradio as gr
import numpy as np
import grpc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

try:
    # Try to import Google Colab-specific modules
    from google_colab_selenium import Chrome
except ImportError:
    # Fall back to regular selenium if not in Colab
    logger.info("Not running in Colab, using standard selenium")
    from selenium import webdriver
    Chrome = webdriver.Chrome

# Import generated modules - these would need to be generated from protobuf
# Fallback implementation if proto files aren't generated
try:
    import story_service_pb2
    import story_service_pb2_grpc
except ImportError:
    logger.warning("Proto modules not found. gRPC functionality will be limited.")
    # Create minimal placeholder classes for demonstration
    class StoryServiceProto:
        class StoryResponse:
            def __init__(self, story="", status=0, message=""):
                self.story = story
                self.status = status
                self.message = message

        class TrendResponse:
            def __init__(self, trends=None, status=0, message=""):
                self.trends = trends or []
                self.status = status
                self.message = message

    # Create minimal module structure
    story_service_pb2 = StoryServiceProto()

    class StoryGeneratorServicer:
        pass

    class Add_StoryGeneratorServicer_to_server:
        def __call__(self, servicer, server):
            pass

    # Mock gRPC module with minimal functionality
    story_service_pb2_grpc = type('', (), {
        'StoryGeneratorServicer': StoryGeneratorServicer,
        'add_StoryGeneratorServicer_to_server': Add_StoryGeneratorServicer_to_server()
    })

# Initialize T5 model and tokenizer
model_name = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()

# Selenium driver setup
def create_driver():
    options = Options()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--incognito")
    options.add_argument("--headless")  # Run in headless mode for server
    return Chrome(options=options)

# Trend scraping functions
def google_trends(region="United States"):
    """Scrape trending topics from Google Trends"""
    try:
        driver = create_driver()
        googlekeywords = []
        driver.get("https://trends.google.com/trends/trendingsearches/daily?geo=US")
        time.sleep(5)

        # Click on region dropdown
        trending_topics = driver.find_element(By.XPATH, "/html/body/c-wiz/div/div[5]/div[1]/c-wiz/div/div[1]/div[1]/div/div[1]/c-wiz/div/div/div[1]/div/button/span[5]")
        trending_topics.click()

        # Enter region
        location = driver.find_element(By.CLASS_NAME, "ODWLre")
        location.send_keys(region + Keys.RETURN)

        # Click on region in dropdown
        clicked = driver.find_element(By.CLASS_NAME, "W7g1Rb-rymPhb-Tkg0ld")
        clicked.click()

        # Extract keywords
        keywords = driver.find_elements(By.CLASS_NAME, "mZ3RIc")
        for i in keywords:
            googlekeywords.append(i.text)

        driver.quit()
        return googlekeywords
    except Exception as e:
        logger.error(f"Error scraping Google Trends: {e}")
        # Return fallback trends if scraping fails
        return ["Technology", "AI", "Climate Change", "Space Exploration", "Quantum Computing"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

def youtube_trends():
    """Scrape trending videos from YouTube"""
    try:
        driver = create_driver()
        youtube_keywords = []
        driver.get("https://www.youtube.com/feed/trending")
        time.sleep(5)

        trendingtopics2 = driver.find_elements(By.ID, "video-title")
        for i in trendingtopics2:
            youtube_keywords.append(i.text)

        # Clean data - remove emojis
        cleaned_data = [emoji.replace_emoji(line, replace='').strip() for line in youtube_keywords]

        driver.quit()
        return cleaned_data
    except Exception as e:
        logger.error(f"Error scraping YouTube Trends: {e}")
        # Return fallback trends if scraping fails
        return ["New Technology Review", "Future of AI", "Space Discovery", "Quantum Computing Explained", "Tech News"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

# Story generation function
def generate_story(trend, max_length=600):
    """Generate a story based on the given trend using T5 model"""
    try:
        prompt = f"Create a detailed and engaging science fiction story about {trend}. The story should take place in a futuristic world where {trend} has significantly impacted society, the environment, and the economy. Explore the physical, social, and environmental impacts of {trend}, including technological advancements, societal changes, and the challenges faced by cities. Discuss the political and economic consequences, focusing on governance, power struggles, and economic shifts. The story should feature compelling characters who are impacted by these changes, a captivating plot that weaves together personal struggles, political intrigue, and technological innovations, and a believable vision of the future. Align the story with the theme of {trend} and create thought-provoking, science fiction elements."

        inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
        story = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return story
    except Exception as e:
        logger.error(f"Error generating story: {e}")
        return f"Error generating story for {trend}. Please try again or choose another trend."

# gRPC Service Implementation
class StoryGeneratorServicer(story_service_pb2_grpc.StoryGeneratorServicer):
    def GenerateStory(self, request, context):
        """Handle story generation requests via gRPC"""
        logger.info(f"Received GenerateStory request for trend: {request.trend}")
        try:
            story = generate_story(request.trend, request.max_length)
            return story_service_pb2.StoryResponse(
                story=story,
                status=200,
                message="Story generated successfully"
            )
        except Exception as e:
            logger.error(f"Error in GenerateStory: {e}")
            return story_service_pb2.StoryResponse(
                story="",
                status=500,
                message=f"Error generating story: {str(e)}"
            )

    def GetTrends(self, request, context):
        """Handle trend retrieval requests via gRPC"""
        logger.info(f"Received GetTrends request for source: {request.source}")
        try:
            if request.source.lower() == "google":
                trends = google_trends(request.region)
            elif request.source.lower() == "youtube":
                trends = youtube_trends()
            else:
                return story_service_pb2.TrendResponse(
                    trends=[],
                    status=400,
                    message="Invalid source. Use 'google' or 'youtube'."
                )

            return story_service_pb2.TrendResponse(
                trends=trends,
                status=200,
                message="Trends retrieved successfully"
            )
        except Exception as e:
            logger.error(f"Error in GetTrends: {e}")
            return story_service_pb2.TrendResponse(
                trends=[],
                status=500,
                message=f"Error retrieving trends: {str(e)}"
            )

# Start gRPC server
def serve_grpc(port=50051):
    """Start the gRPC server on the specified port"""
    try:
        server = grpc.server(ThreadPoolExecutor(max_workers=10))
        story_service_pb2_grpc.add_StoryGeneratorServicer_to_server(
            StoryGeneratorServicer(), server
        )
        server.add_insecure_port(f'[::]:{port}')
        server.start()
        logger.info(f"gRPC server started on port {port}")
        return server
    except Exception as e:
        logger.error(f"Failed to start gRPC server: {e}")
        return None

# Flask App Setup
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    """Flask endpoint for story generation"""
    try:
        data = request.json
        trend = data.get('trend')
        max_length = data.get('max_length', 600)

        if not trend:
            return jsonify({"error": "Missing trend parameter"}), 400

        # Perform generation asynchronously in thread pool
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(generate_story, trend, max_length)
            story = future.result()

        return jsonify({
            "story": story,
            "trend": trend,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /generate endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/trends', methods=['GET'])
def trends_endpoint():
    """Flask endpoint for trend retrieval"""
    try:
        source = request.args.get('source', 'google')
        region = request.args.get('region', 'United States')

        if source.lower() == 'google':
            trends = google_trends(region)
        elif source.lower() == 'youtube':
            trends = youtube_trends()
        else:
            return jsonify({"error": "Invalid source parameter. Use 'google' or 'youtube'"}), 400

        return jsonify({
            "trends": trends,
            "source": source,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /trends endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({"status": "healthy"}), 200

# Start Flask server
def start_flask_app(port=5000):
    """Start the Flask app on the specified port"""
    try:
        logger.info(f"Starting Flask server on port {port}")
        app.run(host='0.0.0.0', port=port)
    except OSError as e:
        # If port is in use, try another port
        if "Address already in use" in str(e):
            new_port = port + 1
            logger.warning(f"Port {port} is in use, trying port {new_port}")
            start_flask_app(new_port)
        else:
            logger.error(f"Error starting Flask server: {e}")

# Gradio Interface
def create_gradio_interface():
    """Create Gradio interface for the application"""
    with gr.Blocks(title="Trend-Based Story Generator") as interface:
        gr.Markdown("# 🚀 Trend-Based Story Generator")
        gr.Markdown("Generate science fiction stories based on current trending topics")

        with gr.Row():
            with gr.Column(scale=1):
                source = gr.Radio(
                    ["Google", "YouTube"],
                    label="Trend Source",
                    value="Google",
                    info="Select where to scrape trending topics from"
                )
                region = gr.Textbox(
                    value="United States",
                    label="Region (Google Trends Only)",
                    info="Enter a region for Google Trends"
                )
                get_trends_btn = gr.Button("Get Trending Topics")

                trend_dropdown = gr.Dropdown(
                    label="Select a Trending Topic",
                    info="Choose a trend to generate a story about",
                    interactive=True,
                    choices=["Technology", "AI", "Climate Change"]  # Default choices
                )

                max_length = gr.Slider(
                    minimum=100,
                    maximum=1000,
                    value=600,
                    step=50,
                    label="Maximum Story Length"
                )

                generate_btn = gr.Button("Generate Story")

            with gr.Column(scale=2):
                output_story = gr.Textbox(
                    label="Generated Story",
                    lines=20,
                    max_lines=30
                )

        # Function to get trends
        def fetch_trends(source, region):
            if source.lower() == "google":
                trends = google_trends(region)
            else:
                trends = youtube_trends()
            # Return choices without using update
            return gr.update(choices=trends, value=trends[0] if trends else None)

        # Function to generate story
        def generate_story_ui(trend, max_length):
            if not trend:
                return "Please select a trend first."
            return generate_story(trend, max_length)

        # Set up event handlers
        get_trends_btn.click(
            fetch_trends,
            inputs=[source, region],
            outputs=[trend_dropdown]
        )

        generate_btn.click(
            generate_story_ui,
            inputs=[trend_dropdown, max_length],
            outputs=[output_story]
        )

    return interface

def main():
    # Start the gRPC server in a separate thread - without ngrok
    grpc_thread = threading.Thread(target=serve_grpc)
    grpc_thread.daemon = True
    grpc_thread.start()

    # Start Flask in a separate thread - without ngrok
    flask_thread = threading.Thread(target=start_flask_app)
    flask_thread.daemon = True
    flask_thread.start()

    # Create and launch Gradio interface
    interface = create_gradio_interface()
    interface.launch(share=True, inline=False)

    # Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        logger.info("Shutting down services...")

if __name__ == "__main__":
    main()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c7de52ac79cdc2d37.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Practice**

In [ ]:
import os
import time
import emoji
import asyncio
import json
import threading
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from flask import Flask, request, jsonify
import gradio as gr
import numpy as np
import grpc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

try:
    # Try to import Google Colab-specific modules
    from google_colab_selenium import Chrome
except ImportError:
    # Fall back to regular selenium if not in Colab
    logger.info("Not running in Colab, using standard selenium")
    from selenium import webdriver
    Chrome = webdriver.Chrome

# Import generated modules - these would need to be generated from protobuf
# Fallback implementation if proto files aren't generated
try:
    import story_service_pb2
    import story_service_pb2_grpc
except ImportError:
    logger.warning("Proto modules not found. gRPC functionality will be limited.")
    # Create minimal placeholder classes for demonstration
    class StoryServiceProto:
        class StoryResponse:
            def __init__(self, story="", status=0, message=""):
                self.story = story
                self.status = status
                self.message = message

        class TrendResponse:
            def __init__(self, trends=None, status=0, message=""):
                self.trends = trends or []
                self.status = status
                self.message = message

    # Create minimal module structure
    story_service_pb2 = StoryServiceProto()

    class StoryGeneratorServicer:
        pass

    class Add_StoryGeneratorServicer_to_server:
        def __call__(self, servicer, server):
            pass

    # Mock gRPC module with minimal functionality
    story_service_pb2_grpc = type('', (), {
        'StoryGeneratorServicer': StoryGeneratorServicer,
        'add_StoryGeneratorServicer_to_server': Add_StoryGeneratorServicer_to_server()
    })

# Initialize T5 model and tokenizer
model_name = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()

# Selenium driver setup
def create_driver():
    options = Options()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--incognito")
    options.add_argument("--headless")  # Run in headless mode for server
    return Chrome(options=options)

# Trend scraping functions
def google_trends(region="United States"):
    """Scrape trending topics from Google Trends"""
    try:
        driver = create_driver()
        googlekeywords = []
        driver.get("https://trends.google.com/trends/trendingsearches/daily?geo=US")
        time.sleep(5)

        # Click on region dropdown
        trending_topics = driver.find_element(By.XPATH, "/html/body/c-wiz/div/div[5]/div[1]/c-wiz/div/div[1]/div[1]/div/div[1]/c-wiz/div/div/div[1]/div/button/span[5]")
        trending_topics.click()

        # Enter region
        location = driver.find_element(By.CLASS_NAME, "ODWLre")
        location.send_keys(region + Keys.RETURN)

        # Click on region in dropdown
        clicked = driver.find_element(By.CLASS_NAME, "W7g1Rb-rymPhb-Tkg0ld")
        clicked.click()

        # Extract keywords
        keywords = driver.find_elements(By.CLASS_NAME, "mZ3RIc")
        for i in keywords:
            googlekeywords.append(i.text)

        driver.quit()
        return googlekeywords
    except Exception as e:
        logger.error(f"Error scraping Google Trends: {e}")
        # Return fallback trends if scraping fails
        return ["Technology", "AI", "Climate Change", "Space Exploration", "Quantum Computing"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

def youtube_trends():
    """Scrape trending videos from YouTube"""
    try:
        driver = create_driver()
        youtube_keywords = []
        driver.get("https://www.youtube.com/feed/trending")
        time.sleep(5)

        trendingtopics2 = driver.find_elements(By.ID, "video-title")
        for i in trendingtopics2:
            youtube_keywords.append(i.text)

        # Clean data - remove emojis
        cleaned_data = [emoji.replace_emoji(line, replace='').strip() for line in youtube_keywords]

        driver.quit()
        return cleaned_data
    except Exception as e:
        logger.error(f"Error scraping YouTube Trends: {e}")
        # Return fallback trends if scraping fails
        return ["New Technology Review", "Future of AI", "Space Discovery", "Quantum Computing Explained", "Tech News"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

# Story generation function
def generate_story(trend, max_length=600):
    """Generate a story based on the given trend using T5 model"""
    try:
        prompt = f"Create a detailed and engaging science fiction story about {trend}. The story should take place in a futuristic world where {trend} has significantly impacted society, the environment, and the economy. Explore the physical, social, and environmental impacts of {trend}, including technological advancements, societal changes, and the challenges faced by cities. Discuss the political and economic consequences, focusing on governance, power struggles, and economic shifts. The story should feature compelling characters who are impacted by these changes, a captivating plot that weaves together personal struggles, political intrigue, and technological innovations, and a believable vision of the future. Align the story with the theme of {trend} and create thought-provoking, science fiction elements."

        inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
        story = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return story
    except Exception as e:
        logger.error(f"Error generating story: {e}")
        return f"Error generating story for {trend}. Please try again or choose another trend."

# gRPC Service Implementation
class StoryGeneratorServicer(story_service_pb2_grpc.StoryGeneratorServicer):
    def GenerateStory(self, request, context):
        """Handle story generation requests via gRPC"""
        logger.info(f"Received GenerateStory request for trend: {request.trend}")
        try:
            story = generate_story(request.trend, request.max_length)
            return story_service_pb2.StoryResponse(
                story=story,
                status=200,
                message="Story generated successfully"
            )
        except Exception as e:
            logger.error(f"Error in GenerateStory: {e}")
            return story_service_pb2.StoryResponse(
                story="",
                status=500,
                message=f"Error generating story: {str(e)}"
            )

    def GetTrends(self, request, context):
        """Handle trend retrieval requests via gRPC"""
        logger.info(f"Received GetTrends request for source: {request.source}")
        try:
            if request.source.lower() == "google":
                trends = google_trends(request.region)
            elif request.source.lower() == "youtube":
                trends = youtube_trends()
            else:
                return story_service_pb2.TrendResponse(
                    trends=[],
                    status=400,
                    message="Invalid source. Use 'google' or 'youtube'."
                )

            return story_service_pb2.TrendResponse(
                trends=trends,
                status=200,
                message="Trends retrieved successfully"
            )
        except Exception as e:
            logger.error(f"Error in GetTrends: {e}")
            return story_service_pb2.TrendResponse(
                trends=[],
                status=500,
                message=f"Error retrieving trends: {str(e)}"
            )

# Start gRPC server
def serve_grpc(port=50051):
    """Start the gRPC server on the specified port"""
    try:
        server = grpc.server(ThreadPoolExecutor(max_workers=10))
        story_service_pb2_grpc.add_StoryGeneratorServicer_to_server(
            StoryGeneratorServicer(), server
        )
        server.add_insecure_port(f'[::]:{port}')
        server.start()
        logger.info(f"gRPC server started on port {port}")
        return server
    except Exception as e:
        logger.error(f"Failed to start gRPC server: {e}")
        return None

# Flask App Setup
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    """Flask endpoint for story generation"""
    try:
        data = request.json
        trend = data.get('trend')
        max_length = data.get('max_length', 600)

        if not trend:
            return jsonify({"error": "Missing trend parameter"}), 400

        # Perform generation asynchronously in thread pool
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(generate_story, trend, max_length)
            story = future.result()

        return jsonify({
            "story": story,
            "trend": trend,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /generate endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/trends', methods=['GET'])
def trends_endpoint():
    """Flask endpoint for trend retrieval"""
    try:
        source = request.args.get('source', 'google')
        region = request.args.get('region', 'United States')

        if source.lower() == 'google':
            trends = google_trends(region)
        elif source.lower() == 'youtube':
            trends = youtube_trends()
        else:
            return jsonify({"error": "Invalid source parameter. Use 'google' or 'youtube'"}), 400

        return jsonify({
            "trends": trends,
            "source": source,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /trends endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({"status": "healthy"}), 200

# Start Flask server
def start_flask_app(port=5000):
    """Start the Flask app on the specified port"""
    try:
        logger.info(f"Starting Flask server on port {port}")

        # Print local access URLs for Flask
        local_url = f"http://127.0.0.1:{port}"
        logger.info("=" * 80)
        logger.info(f"FLASK API LOCAL URL: {local_url}")
        logger.info(f"API ENDPOINTS: {local_url}/trends?source=google")
        logger.info(f"              {local_url}/generate (POST)")
        logger.info(f"              {local_url}/health")
        logger.info("=" * 80)

        app.run(host='0.0.0.0', port=port)
    except OSError as e:
        # If port is in use, try another port
        if "Address already in use" in str(e):
            new_port = port + 1
            logger.warning(f"Port {port} is in use, trying port {new_port}")
            start_flask_app(new_port)
        else:
            logger.error(f"Error starting Flask server: {e}")

# Gradio Interface
def create_gradio_interface():
    """Create Gradio interface for the application"""
    with gr.Blocks(title="Trend-Based Story Generator") as interface:
        gr.Markdown("# 🚀 Trend-Based Story Generator")
        gr.Markdown("Generate science fiction stories based on current trending topics")

        with gr.Row():
            with gr.Column(scale=1):
                source = gr.Radio(
                    ["Google", "YouTube"],
                    label="Trend Source",
                    value="Google",
                    info="Select where to scrape trending topics from"
                )
                region = gr.Textbox(
                    value="United States",
                    label="Region (Google Trends Only)",
                    info="Enter a region for Google Trends"
                )
                get_trends_btn = gr.Button("Get Trending Topics")

                trend_dropdown = gr.Dropdown(
                    label="Select a Trending Topic",
                    info="Choose a trend to generate a story about",
                    interactive=True,
                    choices=["Technology", "AI", "Climate Change"]  # Default choices
                )

                max_length = gr.Slider(
                    minimum=100,
                    maximum=1000,
                    value=600,
                    step=50,
                    label="Maximum Story Length"
                )

                generate_btn = gr.Button("Generate Story")

            with gr.Column(scale=2):
                output_story = gr.Textbox(
                    label="Generated Story",
                    lines=20,
                    max_lines=30
                )

        # Function to get trends
        def fetch_trends(source, region):
            if source.lower() == "google":
                trends = google_trends(region)
            else:
                trends = youtube_trends()
            # Return choices without using update
            return gr.update(choices=trends, value=trends[0] if trends else None)

        # Function to generate story
        def generate_story_ui(trend, max_length):
            if not trend:
                return "Please select a trend first."
            return generate_story(trend, max_length)

        # Set up event handlers
        get_trends_btn.click(
            fetch_trends,
            inputs=[source, region],
            outputs=[trend_dropdown]
        )

        generate_btn.click(
            generate_story_ui,
            inputs=[trend_dropdown, max_length],
            outputs=[output_story]
        )

    return interface

# Function to display public URLs prominently
def display_public_url(url, service_name="GRADIO"):
    """Display a public URL prominently in the console"""
    border = "=" * 80
    message = f"{service_name} PUBLIC URL: {url}"

    # Print to console with prominent formatting
    print("\n\n")
    print(border)
    print(message)
    print(border)
    print("\n\n")

    # Also log it
    logger.info(border)
    logger.info(message)
    logger.info(border)

def main():
    # Start the gRPC server in a separate thread - without ngrok
    grpc_thread = threading.Thread(target=serve_grpc)
    grpc_thread.daemon = True
    grpc_thread.start()

    # Start Flask in a separate thread - without ngrok
    flask_thread = threading.Thread(target=start_flask_app)
    flask_thread.daemon = True
    flask_thread.start()

    # Create and launch Gradio interface
    interface = create_gradio_interface()
    public_url = interface.launch(share=True, inline=False)

    # Display the public URL prominently
    display_public_url(public_url, "GRADIO UI")

    # Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        logger.info("Shutting down services...")

if __name__ == "__main__":
    main()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62f18e3bdd9398d9f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



GRADIO UI PUBLIC URL: 





<IPython.core.display.Javascript object>

**code for links**

In [61]:
import os
import time
import emoji
import asyncio
import json
import threading
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from flask import Flask, request, jsonify
import gradio as gr
import numpy as np
import grpc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

try:
    # Try to import Google Colab-specific modules
    from google_colab_selenium import Chrome
except ImportError:
    # Fall back to regular selenium if not in Colab
    logger.info("Not running in Colab, using standard selenium")
    from selenium import webdriver
    Chrome = webdriver.Chrome

# Import generated modules - these would need to be generated from protobuf
# Fallback implementation if proto files aren't generated
try:
    import story_service_pb2
    import story_service_pb2_grpc
except ImportError:
    logger.warning("Proto modules not found. gRPC functionality will be limited.")
    # Create minimal placeholder classes for demonstration
    class StoryServiceProto:
        class StoryResponse:
            def __init__(self, story="", status=0, message=""):
                self.story = story
                self.status = status
                self.message = message

        class TrendResponse:
            def __init__(self, trends=None, status=0, message=""):
                self.trends = trends or []
                self.status = status
                self.message = message

    # Create minimal module structure
    story_service_pb2 = StoryServiceProto()

    class StoryGeneratorServicer:
        pass

    class Add_StoryGeneratorServicer_to_server:
        def __call__(self, servicer, server):
            pass

    # Mock gRPC module with minimal functionality
    story_service_pb2_grpc = type('', (), {
        'StoryGeneratorServicer': StoryGeneratorServicer,
        'add_StoryGeneratorServicer_to_server': Add_StoryGeneratorServicer_to_server()
    })

# Initialize T5 model and tokenizer
model_name = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.eval()

# Selenium driver setup
def create_driver():
    options = Options()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--incognito")
    options.add_argument("--headless")  # Run in headless mode for server
    return Chrome(options=options)

# Trend scraping functions
def google_trends(region="United States"):
    """Scrape trending topics from Google Trends"""
    try:
        driver = create_driver()
        googlekeywords = []
        driver.get("https://trends.google.com/trends/trendingsearches/daily?geo=US")
        time.sleep(5)

        # Click on region dropdown
        trending_topics = driver.find_element(By.XPATH, "/html/body/c-wiz/div/div[5]/div[1]/c-wiz/div/div[1]/div[1]/div/div[1]/c-wiz/div/div/div[1]/div/button/span[5]")
        trending_topics.click()

        # Enter region
        location = driver.find_element(By.CLASS_NAME, "ODWLre")
        location.send_keys(region + Keys.RETURN)

        # Click on region in dropdown
        clicked = driver.find_element(By.CLASS_NAME, "W7g1Rb-rymPhb-Tkg0ld")
        clicked.click()

        # Extract keywords
        keywords = driver.find_elements(By.CLASS_NAME, "mZ3RIc")
        for i in keywords:
            googlekeywords.append(i.text)

        driver.quit()
        return googlekeywords
    except Exception as e:
        logger.error(f"Error scraping Google Trends: {e}")
        # Return fallback trends if scraping fails
        return ["Technology", "AI", "Climate Change", "Space Exploration", "Quantum Computing"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

def youtube_trends():
    """Scrape trending videos from YouTube"""
    try:
        driver = create_driver()
        youtube_keywords = []
        driver.get("https://www.youtube.com/feed/trending")
        time.sleep(5)

        trendingtopics2 = driver.find_elements(By.ID, "video-title")
        for i in trendingtopics2:
            youtube_keywords.append(i.text)

        # Clean data - remove emojis
        cleaned_data = [emoji.replace_emoji(line, replace='').strip() for line in youtube_keywords]

        driver.quit()
        return cleaned_data
    except Exception as e:
        logger.error(f"Error scraping YouTube Trends: {e}")
        # Return fallback trends if scraping fails
        return ["New Technology Review", "Future of AI", "Space Discovery", "Quantum Computing Explained", "Tech News"]
    finally:
        if 'driver' in locals():
            try:
                driver.quit()
            except:
                pass

# Story generation function
def generate_story(trend, max_length=600):
    """Generate a story based on the given trend using T5 model"""
    try:
        prompt = f"Create a detailed and engaging science fiction story about {trend}. The story should take place in a futuristic world where {trend} has significantly impacted society, the environment, and the economy. Explore the physical, social, and environmental impacts of {trend}, including technological advancements, societal changes, and the challenges faced by cities. Discuss the political and economic consequences, focusing on governance, power struggles, and economic shifts. The story should feature compelling characters who are impacted by these changes, a captivating plot that weaves together personal struggles, political intrigue, and technological innovations, and a believable vision of the future. Align the story with the theme of {trend} and create thought-provoking, science fiction elements."

        inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
        story = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return story
    except Exception as e:
        logger.error(f"Error generating story: {e}")
        return f"Error generating story for {trend}. Please try again or choose another trend."

# gRPC Service Implementation
class StoryGeneratorServicer(story_service_pb2_grpc.StoryGeneratorServicer):
    def GenerateStory(self, request, context):
        """Handle story generation requests via gRPC"""
        logger.info(f"Received GenerateStory request for trend: {request.trend}")
        try:
            story = generate_story(request.trend, request.max_length)
            return story_service_pb2.StoryResponse(
                story=story,
                status=200,
                message="Story generated successfully"
            )
        except Exception as e:
            logger.error(f"Error in GenerateStory: {e}")
            return story_service_pb2.StoryResponse(
                story="",
                status=500,
                message=f"Error generating story: {str(e)}"
            )

    def GetTrends(self, request, context):
        """Handle trend retrieval requests via gRPC"""
        logger.info(f"Received GetTrends request for source: {request.source}")
        try:
            if request.source.lower() == "google":
                trends = google_trends(request.region)
            elif request.source.lower() == "youtube":
                trends = youtube_trends()
            else:
                return story_service_pb2.TrendResponse(
                    trends=[],
                    status=400,
                    message="Invalid source. Use 'google' or 'youtube'."
                )

            return story_service_pb2.TrendResponse(
                trends=trends,
                status=200,
                message="Trends retrieved successfully"
            )
        except Exception as e:
            logger.error(f"Error in GetTrends: {e}")
            return story_service_pb2.TrendResponse(
                trends=[],
                status=500,
                message=f"Error retrieving trends: {str(e)}"
            )

# Start gRPC server
def serve_grpc(port=50051):
    """Start the gRPC server on the specified port"""
    try:
        server = grpc.server(ThreadPoolExecutor(max_workers=10))
        story_service_pb2_grpc.add_StoryGeneratorServicer_to_server(
            StoryGeneratorServicer(), server
        )
        server.add_insecure_port(f'[::]:{port}')
        server.start()
        logger.info(f"gRPC server started on port {port}")
        return server
    except Exception as e:
        logger.error(f"Failed to start gRPC server: {e}")
        return None

# Flask App Setup
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    """Flask endpoint for story generation"""
    try:
        data = request.json
        trend = data.get('trend')
        max_length = data.get('max_length', 600)

        if not trend:
            return jsonify({"error": "Missing trend parameter"}), 400

        # Perform generation asynchronously in thread pool
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(generate_story, trend, max_length)
            story = future.result()

        return jsonify({
            "story": story,
            "trend": trend,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /generate endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/trends', methods=['GET'])
def trends_endpoint():
    """Flask endpoint for trend retrieval"""
    try:
        source = request.args.get('source', 'google')
        region = request.args.get('region', 'United States')

        if source.lower() == 'google':
            trends = google_trends(region)
        elif source.lower() == 'youtube':
            trends = youtube_trends()
        else:
            return jsonify({"error": "Invalid source parameter. Use 'google' or 'youtube'"}), 400

        return jsonify({
            "trends": trends,
            "source": source,
            "status": "success"
        }), 200
    except Exception as e:
        logger.error(f"Error in /trends endpoint: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({"status": "healthy"}), 200

# Start Flask server
def start_flask_app(port=5000):
    """Start the Flask app on the specified port"""
    try:
        logger.info(f"Starting Flask server on port {port}")

        # Get local IP address
        local_ip = socket.gethostbyname(socket.gethostname())
        local_url = f"http://127.0.0.1:{port}"
        network_url = f"http://{local_ip}:{port}"

        logger.info("=" * 80)
        print("FLASK API LOCAL ACCESS:")
        print(f"  {local_url}")
        print("FLASK API NETWORK ACCESS (on your LAN):")
        print(f"  {network_url}")
        print("\nAPI ENDPOINTS:")
        print(f"  {local_url}/trends?source=google")
        print(f"  {local_url}/generate (POST)")
        print(f"  {local_url}/health")
        print("\nNetwork endpoints:")
        print(f"  {network_url}/trends?source=google")
        print(f"  {network_url}/generate (POST)")
        print(f"  {network_url}/health")
        logger.info("=" * 80)

        app.run(host='0.0.0.0', port=port)
    except OSError as e:
        if "Address already in use" in str(e):
            new_port = port + 1
            logger.warning(f"Port {port} is in use, trying port {new_port}")
            start_flask_app(new_port)
        else:
            logger.error(f"Error starting Flask server: {e}")
# Gradio Interface
def create_gradio_interface():
    """Create Gradio interface for the application"""
    with gr.Blocks(title="Trend-Based Story Generator") as interface:
        gr.Markdown("# 🚀 Trend-Based Story Generator")
        gr.Markdown("Generate science fiction stories based on current trending topics")

        with gr.Row():
            with gr.Column(scale=1):
                source = gr.Radio(
                    ["Google", "YouTube"],
                    label="Trend Source",
                    value="Google",
                    info="Select where to scrape trending topics from"
                )
                region = gr.Textbox(
                    value="United States",
                    label="Region (Google Trends Only)",
                    info="Enter a region for Google Trends"
                )
                get_trends_btn = gr.Button("Get Trending Topics")

                trend_dropdown = gr.Dropdown(
                    label="Select a Trending Topic",
                    info="Choose a trend to generate a story about",
                    interactive=True,
                    choices=["Technology", "AI", "Climate Change"]  # Default choices
                )

                max_length = gr.Slider(
                    minimum=100,
                    maximum=1000,
                    value=600,
                    step=50,
                    label="Maximum Story Length"
                )

                generate_btn = gr.Button("Generate Story")

            with gr.Column(scale=2):
                output_story = gr.Textbox(
                    label="Generated Story",
                    lines=20,
                    max_lines=30
                )

        # Function to get trends
        def fetch_trends(source, region):
            if source.lower() == "google":
                trends = google_trends(region)
            else:
                trends = youtube_trends()
            # Return choices without using update
            return gr.update(choices=trends, value=trends[0] if trends else None)

        # Function to generate story
        def generate_story_ui(trend, max_length):
            if not trend:
                return "Please select a trend first."
            return generate_story(trend, max_length)

        # Set up event handlers
        get_trends_btn.click(
            fetch_trends,
            inputs=[source, region],
            outputs=[trend_dropdown]
        )

        generate_btn.click(
            generate_story_ui,
            inputs=[trend_dropdown, max_length],
            outputs=[output_story]
        )

    return interface

# Function to display public URLs prominently
def display_public_url(url, service_name="GRADIO"):
    """Display a public URL prominently in the console"""
    border = "=" * 80
    message = f"{service_name} PUBLIC URL: {url}"

    # Print to console with prominent formatting
    print("\n\n")
    print(border)
    print(message)
    print(border)
    print("\n\n")

    # Also log it
    logger.info(border)
    logger.info(message)
    logger.info(border)

def main():
    # Start the gRPC server in a separate thread - without ngrok
    grpc_thread = threading.Thread(target=serve_grpc)
    grpc_thread.daemon = True
    grpc_thread.start()

    # Start Flask in a separate thread - without ngrok
    flask_thread = threading.Thread(target=start_flask_app)
    flask_thread.daemon = True
    flask_thread.start()

    # Create and launch Gradio interface
    interface = create_gradio_interface()
    public_url = interface.launch(share=True, inline=False)

    # Display the public URL prominently
    display_public_url(public_url, "GRADIO UI")

    # Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        logger.info("Shutting down services...")

if __name__ == "__main__":
    main()

FLASK API LOCAL ACCESS:
  http://127.0.0.1:5000
FLASK API NETWORK ACCESS (on your LAN):
  http://172.28.0.12:5000

API ENDPOINTS:
  http://127.0.0.1:5000/trends?source=google
  http://127.0.0.1:5000/generate (POST)
  http://127.0.0.1:5000/health

Network endpoints:
  http://172.28.0.12:5000/trends?source=google
  http://172.28.0.12:5000/generate (POST)
  http://172.28.0.12:5000/health
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://629b2c4059ddc88da8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



GRADIO UI PUBLIC URL: 



